In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

In [2]:
df = pd.read_csv('../data/data_modeling.csv')

## Train/Test split 

In [3]:
X_major = df.drop(columns='sub_class')
y_major = df['sub_class']
X_train, X_val, y_train, y_val = train_test_split(X_major, y_major, random_state=42, stratify=y_major)

## Model: Random Forest

In [4]:
features_to_drop = ['body', 'score', 'id', 'word_count', 'count_char',
       'word_count_sqrt', 'count_char_sqrt', 'neg', 'neu', 'pos']

X_train_rf = X_train.drop(columns=features_to_drop)
X_val_rf = X_val.drop(columns=features_to_drop)

y_train_rf = y_train
y_val_rf = y_val

### Baseline Accuracy

In [5]:
df['sub_class'].value_counts(normalize=True)

1    0.543779
0    0.456221
Name: sub_class, dtype: float64

### Model implementation

In [6]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, stratify=y)

In [7]:
rf = RandomForestClassifier()

In [8]:
rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3, 4, 5],
}
gs = GridSearchCV(rf, param_grid=rf_params, cv=5)
gs.fit(X_train_rf, y_train_rf)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 1, 2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]})

In [9]:
gs.score(X_train_rf, y_train_rf)

0.98224043715847

In [10]:
gs.score(X_val_rf, y_val_rf)

0.7034764826175869

In [50]:
gs.set_params

<bound method BaseEstimator.set_params of GridSearchCV(estimator=SVC(),
             param_grid={'C': array([1.00000000e-03, 5.27263158e-01, 1.05352632e+00, 1.57978947e+00,
       2.10605263e+00, 2.63231579e+00, 3.15857895e+00, 3.68484211e+00,
       4.21110526e+00, 4.73736842e+00, 5.26363158e+00, 5.78989474e+00,
       6.31615789e+00, 6.84242105e+00, 7.36868421e+00, 7.89494737e+00,
       8.42121053e+00, 8.94747368e+00, 9.47373684e+00, 1.00000000e+01])})>

## Model: Logisitic Regression on numerical data

In [11]:
features = ['score', 'word_count', 'count_char',
            'word_count_sqrt', 'count_char_sqrt',
            'neg','neu', 'pos', 'compound' ]

X_train_logreg = X_train[features]
X_val_logreg = X_val[features]

y_train_logreg = y_train
y_val_logreg = y_val

In [12]:
X_train_logreg.dtypes

score                int64
word_count           int64
count_char           int64
word_count_sqrt    float64
count_char_sqrt    float64
neg                float64
neu                float64
pos                float64
compound           float64
dtype: object

In [13]:
X_train_logreg.isna().sum()

score              0
word_count         0
count_char         0
word_count_sqrt    0
count_char_sqrt    0
neg                0
neu                0
pos                0
compound           0
dtype: int64

In [14]:
params= {
    "C":[0.001,0.01,0.1,1,10,100],
    'solver':['newton-cg', 'lbfgs', 'sag'], 
    "penalty":['l2']
}
logreg = LogisticRegression( max_iter=8_000)
logreg_cv = GridSearchCV(logreg,params,cv=5)
logreg_cv.fit(X_train_logreg,y_train_logreg)

/usr/local/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/usr/local/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=8000),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'sag']})

In [15]:
logreg_cv.best_params_

{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}

In [16]:
logreg_cv.score(X_train_logreg, y_train_logreg)

0.6734972677595629

In [17]:
logreg_cv.score(X_val_logreg, y_val_logreg)

0.656441717791411

## Model: Logisitic Regression on vectorized text

In [19]:
X_train_logreg2 = X_train_rf
X_val_logreg2 = X_val_rf 

y_train_logreg2 = y_train_rf 
y_val_logreg2 = y_val_rf 

In [21]:
params= {
    "C":[0.001,0.01,0.1,1,10,100],
    'solver':['newton-cg'], 
    "penalty":['l2']
}
logreg2 = LogisticRegression(max_iter=4_000)
logreg2_cv = GridSearchCV(logreg2,params,cv=3)
logreg2_cv.fit(X_train_logreg2 ,y_train_logreg2)

GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=4000),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2'],
                         'solver': ['newton-cg']})

In [23]:
logreg2_cv.score(X_train_logreg2, y_train_logreg2)

0.8920765027322405

In [24]:
logreg2_cv.score(X_val_logreg2, y_val_logreg2)

0.7341513292433538

In [47]:
logreg2_cv.best_params_

{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}

## Model: SVM on vectorized text data

In [29]:
X_train_svm = X_train_rf
X_val_svm = X_val_rf 

y_train_svm = y_train_rf 
y_val_svm = y_val_rf 

In [60]:
svm = SVC(C=1.5)
svm.fit(X_train_svm, y_train_svm)

SVC(C=1.5)

In [58]:
svm.score(X_train_svm, y_train_svm)

0.9275956284153005

In [61]:
svm.score(X_val_svm, y_val_svm)

0.7116564417177914

## Model: SVM on vectorized text data + Feature Selection [DINT]

In [123]:
svm_f = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False, max_iter=4_000))),
  ('classification', SVC())
])
svm_f.fit(X_train_svm, y_train_svm)

Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False, max_iter=4000,
                                                     penalty='l1'))),
                ('classification', SVC())])

In [124]:
svm_f.score(X_train_svm, y_train_svm)

0.9077868852459017

In [125]:
svm_f.score(X_val_svm, y_val_svm)

0.6871165644171779

## Model: Randorm Forest vectorized text + VADER

In [18]:
features_to_drop = ['body', 'id', 'word_count', 'count_char']

X_train_rfc = X_train.drop(columns=features_to_drop)
X_val_rfc = X_val.drop(columns=features_to_drop)

y_train_rfc = y_train
y_val_rfc = y_val

In [19]:
rfc = RandomForestClassifier()
rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3, 4, 5],
}
gs = GridSearchCV(rfc, param_grid=rf_params, cv=5)
gs.fit(X_train_rfc, y_train_rfc)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 1, 2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]})

In [22]:
gs.score(X_train_rfc, y_train_rfc)

0.9979508196721312

In [23]:
gs.score(X_val_rfc, y_val_rfc)

0.7259713701431493